# Convolution Neural Network with Pytorch Lightning

In [1]:
from __future__ import annotations
import lightning.pytorch as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics

In [4]:
class LeNet(pl.LightningModule):
    def __init__(self, in_channels: int, out_channels: int, lr: float=2e-4):
        """
        Args:
        - in_channels: 1 for grayscale input image, 3 for RGB image
        - out_channels: Number of classes of the classifier
        """
        super.__init__()
        # Debugging tool to display intermediate input/output size of your layer (called before fit)
        self.example_input_array = torch.Tensor(16, in_channels, 32, 32)
        self.learning_rate = lr
        
        # torchmetrics calculate the Accuracy
        self.train_accuracy = torchmetrics.Accuracy(task="multclass", num_classes=out_channels)
        self.val_accuracy = torchmetrics.Accuracy(task="multclass", num_classes=out_channels)
        self.test_accuracy = torchmetrics.Accuracy(task="multclass", num_classes=out_channels)
        
        # Convolution with 5*5 kernel + 2 padding
        # [img_size] 32 -> conv -> 32 -> (max pool) -> 16
        # with 6 output activation maps
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(
                in_channels = in_channels,
                out_channels=6,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.MaxPool2d(kernel_size=2),
        )
        
        # Convolution with 5*5 Kernel (no pad)
        # Pool with 2*2 Max Kernel + 2 Stride
        self.conv_layer2 = nn.Sequential(
            nn.Con2d(
                in_channels=6,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=0
            ),
            nn.MaxPool2d(kernel_size=2),
        )
        
        #FC Layer
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        # "Softmax" Layer
        self.fc3 = nn.Linear(in_features=84, out_features=out_channels)
        
    def forword(self, x:torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv_layer1(x))
        x = F.relu(self.conv_layer2(x))
        # flatten
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # Implementing the training, validation, and test steps
    def configure_optimizers(self) -> torch.optim.Adam:
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

1.21.5
